# 3 register business logic

change ${CORE_ROOT} to your path of `core`.

In [ ]:
export CORE_ROOT="${HOME}/core"

change ${PJ_ROOT} to your path of `uoa-poc2`.

In [ ]:
export PJ_ROOT="${HOME}/uoa-poc2"
cd ${PJ_ROOT};pwd

example)
```
/Users/user/uoa-poc2
```

## load environment variables

load from `core`

In [ ]:
source ${CORE_ROOT}/docs/environments/azure_aks/env

load from `uoa-poc2`

In [ ]:
source ${PJ_ROOT}/docs/environments/azure_aks/env

## setup alias

In [ ]:
alias now="python -c 'import datetime; print(datetime.datetime.now().strftime(\"%Y-%m-%dT%H:%M:%S.%f+09:00\"))'"
alias iso8601="python -c 'import datetime; print(datetime.datetime.now(tz=datetime.timezone.utc).isoformat(timespec=\"seconds\"))'"

## setup entities for business logic

### delete existing `place` entities

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
EXISTING_PLACES=$(curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DELIVERY_ROBOT_SERVICEPATH}" "https://api.${DOMAIN}/orion/v2/entities/?type=place&limit=1000&attrs=id")
LEN=$(echo ${EXISTING_PLACES} | jq length)
count=0
if [ ${LEN} -gt 0 ]; then
  for i in $(seq 0 $((${LEN} - 1))); do
    id=$(echo ${EXISTING_PLACES} | jq .[${i}].id -r)
    echo "${id} will be deleted."
    curl -i -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DELIVERY_ROBOT_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/${id}/?type=place -X DELETE
    let ++count
  done
fi
echo "${count} entities were deleted."

example)

```
place_A will be deleted.
HTTP/1.1 204 No Content
content-length: 0
fiware-correlator: 6f62f00c-01e4-11ea-8c70-aaa36e462053
date: Fri, 08 Nov 2019 04:58:41 GMT
x-envoy-upstream-service-time: 4
server: envoy

place_B will be deleted.
HTTP/1.1 204 No Content
content-length: 0
fiware-correlator: 6f742ca0-01e4-11ea-b2b0-3e52937c8752
date: Fri, 08 Nov 2019 04:58:41 GMT
x-envoy-upstream-service-time: 4
server: envoy

...
place_RC will be deleted.
HTTP/1.1 204 No Content
content-length: 0
fiware-correlator: 724ef75c-01e4-11ea-8f75-2621012d3c7f
date: Fri, 08 Nov 2019 04:58:46 GMT
x-envoy-upstream-service-time: 4
server: envoy

31 entities were deleted.
```

### register `place` entities

In [ ]:
PLACES=$(cat << __EOD__
[
  {
    "id": "place_A", "name": "受付前",
    "pose": {
      "point": {"x": 0.0, "y": 0.0, "z": 0.0},
      "angle":{"roll": 0.0, "pitch": 0.0, "yaw": 0.0}
    }
  },
  {
    "id": "place_B", "name": "中央経由地点",
    "pose": {
      "point": {"x": -1.50, "y": -2.50, "z": 0.0},
      "angle":{"roll": 0.0, "pitch": 0.0, "yaw": 0.0}
    }
  },
  {
    "id": "place_C", "name": "エレベータ前",
    "pose": {
      "point": {"x": -1.38, "y": 4.97, "z": 0.0},
      "angle":{"roll": 0.0, "pitch": 0.0, "yaw": 1.57}
    }
  },
  {
    "id": "place_D", "name": "玄関",
    "pose": {
      "point": {"x": -4.98, "y": -1.51, "z": 0.0},
      "angle":{"roll": 0.0, "pitch": 0.0, "yaw": 3.12}
    }
  },
  {
    "id": "place_E", "name": "カフェ前",
    "pose": {
      "point": {"x": -1.66, "y": -6.68, "z": 0.0},
      "angle":{"roll": 0.0, "pitch": 0.0, "yaw": -1.97}
    }
  },
  {
    "id": "place_F", "name": "個室前経由地",
    "pose": {
      "point": {"x": 0.20, "y": -8.10, "z": 0.0},
      "angle":{"roll": 0.0, "pitch": 0.0, "yaw": 0.0}
    }
  },
  {
    "id": "place_G", "name": "個室",
    "pose": {
      "point": {"x": 1.71, "y": -9.70, "z": 0.0},
      "angle":{"roll": 0.0, "pitch": 0.0, "yaw": -1.57}
    }
  },
  {
    "id": "place_H", "name": "会議室3",
    "pose": {
      "point": {"x": 6.06, "y": -8.00, "z": 0.0},
      "angle":{"roll": 0.0, "pitch": 0.0, "yaw": -1.57}
    }
  },
  {
    "id": "place_Hdash", "name": "会議室3中",
    "pose": {
      "point": {"x": 5.96, "y": -9.09, "z": 0.0},
      "angle":{"roll": 0.0, "pitch": 0.0, "yaw": -1.57}
    }
  },
  {
    "id": "place_I", "name": "会議室2",
    "pose": {
      "point": {"x": 9.56, "y": -8.00, "z": 0.0},
      "angle":{"roll": 0.0, "pitch": 0.0, "yaw": -1.57}
    }
  },
  {
    "id": "place_Idash", "name": "会議室2中",
    "pose": {
      "point": {"x": 9.43, "y": -9.10, "z": 0.0},
      "angle":{"roll": 0.0, "pitch": 0.0, "yaw": -1.57}
    }
  },
  {
    "id": "place_J", "name": "会議室1",
    "pose": {
      "point": {"x": 10.60, "y": -8.00, "z": 0.0},
      "angle":{"roll": 0.0, "pitch": 0.0, "yaw": -1.57}
    }
  },
  {
    "id": "place_Jdash", "name": "会議室1中",
    "pose": {
      "point": {"x": 10.73, "y": -9.08, "z": 0.0},
      "angle":{"roll": 0.0, "pitch": 0.0, "yaw": -1.57}
    }
  },
  {
    "id": "place_K", "name": "ホワイトボードルーム",
    "pose": {
      "point": {"x": 13.49, "y": -7.90, "z": 0.0},
      "angle":{"roll": 0.0, "pitch": 0.0, "yaw": 0.0}
    }
  },
  {
    "id": "place_L", "name": "事務室",
    "pose": {
      "point": {"x": 4.71, "y": -1.99, "z": 0.0},
      "angle":{"roll": 0.0, "pitch": 0.0, "yaw": -1.57}
    }
  },
  {
    "id": "place_M", "name": "倉庫1",
    "pose": {
      "point": {"x": 8.77, "y": -1.97, "z": 0.0},
      "angle":{"roll": 0.0, "pitch": 0.0, "yaw": -1.57}
    }
  },
  {
    "id": "place_N", "name": "セキュリティルーム",
    "pose": {
      "point": {"x": 10.84, "y": -1.87, "z": 0.0},
      "angle":{"roll": 0.0, "pitch": 0.0, "yaw": 1.57}
    }
  },
  {
    "id": "place_O", "name": "トイレ前",
    "pose": {
      "point": {"x": 13.78, "y": -1.89, "z": 0.0},
      "angle":{"roll": 0.0, "pitch": 0.0, "yaw": -1.57}
    }
  },
  {
    "id": "place_P", "name": "裏口",
    "pose": {
      "point": {"x": -14.80, "y": -1.90, "z": 0.0},
      "angle":{"roll": 0.0, "pitch": 0.0, "yaw": 0.0}
    }
  },
  {
    "id": "place_Q", "name": "待機場所１",
    "pose": {
      "point": {"x": -0.40, "y": 1.50, "z": 0.0},
      "angle":{"roll": 0.0, "pitch": 0.0, "yaw": -1.57}
    }
  },
  {
    "id": "place_R", "name": "中央経由地点２",
    "pose": {
      "point": {"x": 0.50, "y": -2.00, "z": 0.0},
      "angle":{"roll": 0.0, "pitch": 0.0, "yaw": 0.0}
    }
  },
  {
    "id": "place_S", "name": "カフェ前２",
    "pose": {
      "point": {"x": -0.94, "y": -5.90, "z": 0.0},
      "angle":{"roll": 0.0, "pitch": 0.0, "yaw": 0.0}
    }
  },
  {
    "id": "place_T", "name": "個室前経由地２",
    "pose": {
      "point": {"x": 1.70, "y": -8.00, "z": 0.0},
      "angle":{"roll": 0.0, "pitch": 0.0, "yaw": 0.0}
    }
  },
  {
    "id": "place_U", "name": "一時待機場所",
    "pose": {
      "point": {"x": 5.80, "y": -7.15, "z": 0.0},
      "angle":{"roll": 0.0, "pitch": 0.0, "yaw": -1.57}
    }
  },
  {
    "id": "place_V", "name": "待機場所２",
    "pose": {
      "point": {"x": -1.50, "y": 1.50, "z": 0.0},
      "angle":{"roll": 0.0, "pitch": 0.0, "yaw": -1.57}
    }
  },
  {
    "id": "place_LA", "name": "事務室Token取得ポイント",
    "pose": {
      "point": {"x": -0.50, "y": -1.50, "z": 0.0},
      "angle":{"roll": 0.0, "pitch": 0.0, "yaw": -0.46}
    }
  },
  {
    "id": "place_RA", "name": "事務室Token解放ポイント",
    "pose": {
      "point": {"x": -0.50, "y": -2.50, "z": 0.0},
      "angle":{"roll": 0.0, "pitch": 0.0, "yaw": -1.80}
    }
  },
  {
    "id": "place_LB", "name": "個室Token取得ポイント",
    "pose": {
      "point": {"x": 0.70, "y": -7.40, "z": 0.0},
      "angle":{"roll": 0.0, "pitch": 0.0, "yaw": -0.15}
    }
  },
  {
    "id": "place_RB", "name": "個室Token解放ポイント",
    "pose": {
      "point": {"x": 2.30, "y": -7.50, "z": 0.0},
      "angle":{"roll": 0.0, "pitch": 0.0, "yaw": 0.05}
    }
  },
  {
    "id": "place_LC", "name": "会議室Token取得ポイント",
    "pose": {
      "point": {"x": 4.90, "y": -7.30, "z": 0.0},
      "angle":{"roll": 0.0, "pitch": 0.0, "yaw": 0.00}
    }
  },
  {
    "id": "place_RC", "name": "会議室Token解放ポイント",
    "pose": {
      "point": {"x": 5.00, "y": -8.10, "z": 0.0},
      "angle":{"roll": 0.0, "pitch": 0.0, "yaw": 3.14}
    }
  }
]
__EOD__
)

TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
LEN=$(echo ${PLACES} | jq length)
count=0
if [ ${LEN} -gt 0 ]; then
  for i in $(seq 0 $((${LEN} - 1))); do
    id=$(echo ${PLACES} | jq .[${i}].id)
    name=$(echo ${PLACES} | jq .[${i}].name)
    pose=$(echo ${PLACES} | jq .[${i}].pose)
    echo "${id} (${name}) will be registerd."
    curl -i -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DELIVERY_ROBOT_SERVICEPATH}" -H "Content-Type: application/json" https://api.${DOMAIN}/orion/v2/entities/ -X POST -d @-<<__EOS__
{
  "id": ${id},
  "type": "place",
  "TimeInstant": {
    "type": "ISO8601",
    "value": "$(iso8601)"
  },
  "name": {
    "type": "string",
    "value": ${name}
  },
  "pose": {
    "type": "object",
    "value": ${pose}
  }
}
__EOS__
    let ++count
  done
fi
echo "${count} entities were registered."

example)

```
"place_A" ("受付前") will be registerd.
HTTP/1.1 201 Created
content-length: 0
location: /v2/entities/place_A?type=place
fiware-correlator: 950aa66a-01e4-11ea-b2b0-3e52937c8752
date: Fri, 08 Nov 2019 04:59:44 GMT
x-envoy-upstream-service-time: 6
server: envoy

"place_B" ("中央経由地点") will be registerd.
HTTP/1.1 201 Created
content-length: 0
location: /v2/entities/place_B?type=place
fiware-correlator: 95442d22-01e4-11ea-acd2-aaa36e462053
date: Fri, 08 Nov 2019 04:59:45 GMT
x-envoy-upstream-service-time: 8
server: envoy

...
"place_RC" ("会議室Token解放ポイント") will be registerd.
HTTP/1.1 201 Created
content-length: 0
location: /v2/entities/place_RC?type=place
fiware-correlator: 9b01e9fc-01e4-11ea-b2b0-3e52937c8752
date: Fri, 08 Nov 2019 04:59:55 GMT
x-envoy-upstream-service-time: 6
server: envoy

31 entities were registered.
```

### confirm registered `place` entities

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DELIVERY_ROBOT_SERVICEPATH}" "https://api.${DOMAIN}/orion/v2/entities/?type=place&limit=1000" | jq .

example)

```json
[
  {
    "id": "place_A",
    "type": "place",
    "TimeInstant": {
      "type": "ISO8601",
      "value": "2019-11-08T04:59:45.00Z",
      "metadata": {}
    },
    "name": {
      "type": "string",
      "value": "受付前",
      "metadata": {}
    },
    "pose": {
      "type": "object",
      "value": {
        "point": {
          "x": 0,
          "y": 0,
          "z": 0
        },
        "angle": {
          "roll": 0,
          "pitch": 0,
          "yaw": 0
        }
      },
      "metadata": {}
    }
  },
  {
    "id": "place_B",
    "type": "place",
    "TimeInstant": {
      "type": "ISO8601",
      "value": "2019-11-08T04:59:45.00Z",
      "metadata": {}
    },
    "name": {
      "type": "string",
      "value": "中央経由地点",
      "metadata": {}
    },
    "pose": {
      "type": "object",
      "value": {
        "point": {
          "x": -1,
          "y": -2.5,
          "z": 0
        },
        "angle": {
          "roll": 0,
          "pitch": 0,
          "yaw": 0
        }
      },
      "metadata": {}
    }
  },

...

  {
    "id": "place_RC",
    "type": "place",
    "TimeInstant": {
      "type": "ISO8601",
      "value": "2019-11-08T04:59:55.00Z",
      "metadata": {}
    },
    "name": {
      "type": "string",
      "value": "会議室Token解放ポイント",
      "metadata": {}
    },
    "pose": {
      "type": "object",
      "value": {
        "point": {
          "x": 4,
          "y": -8,
          "z": 0
        },
        "angle": {
          "roll": 0,
          "pitch": 0,
          "yaw": 3.14
        }
      },
      "metadata": {}
    }
  }
]
```

### delete existing `route_plan` entities

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
EXISTING_ROUTE_PLANS=$(curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DELIVERY_ROBOT_SERVICEPATH}" "https://api.${DOMAIN}/orion/v2/entities/?type=route_plan&limit=1000&attrs=id")
LEN=$(echo ${EXISTING_ROUTE_PLANS} | jq length)
count=0
if [ ${LEN} -gt 0 ]; then
  for i in $(seq 0 $((${LEN} - 1))); do
    id=$(echo ${EXISTING_ROUTE_PLANS} | jq .[${i}].id -r)
    echo "${id} will be deleted."
    curl -i -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DELIVERY_ROBOT_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/${id}/?type=route_plan -X DELETE
    let ++count
  done
fi
echo "${count} entities were deleted."

example)
```
route_plan_01_01 will be deleted.
HTTP/1.1 204 No Content
content-length: 0
fiware-correlator: e699169c-01e4-11ea-bfd1-2621012d3c7f
date: Fri, 08 Nov 2019 05:02:01 GMT
x-envoy-upstream-service-time: 5
server: envoy

route_plan_01_02 will be deleted.
HTTP/1.1 204 No Content
content-length: 0
fiware-correlator: e6b87f00-01e4-11ea-acd2-aaa36e462053
date: Fri, 08 Nov 2019 05:02:01 GMT
x-envoy-upstream-service-time: 5
server: envoy

...
route_plan_09_02 will be deleted.
HTTP/1.1 204 No Content
content-length: 0
fiware-correlator: e828933e-01e4-11ea-8f75-2621012d3c7f
date: Fri, 08 Nov 2019 05:02:04 GMT
x-envoy-upstream-service-time: 4
server: envoy

18 entities were deleted.
```

### register `route_plan` entities

In [ ]:
ROUTE_PLANS=$(cat << __EOD__
[
  {
    "id": "route_plan_01_01",
    "robot_id": "delivery_robot_01",
    "source": "place_Q", "via": "place_L", "destination": "place_Hdash", 
    "routes": [
      {"from": "place_Q", "via": [], "to": "place_LA", "action": {"func": "lock", "token":"token_a", "waiting_route": {}}, "destination": "place_L"},
      {"from": "place_LA", "via": ["place_R"], "to": "place_L", "action": {"func": "", "token": "", "waiting_route": {}}, "destination": "place_L"},
      {"from": "place_L", "via": ["place_R"], "to": "place_RA", "action": {"func": "release", "token": "token_a", "waiting_route": {}}, "destination": "place_Hdash"},
      {"from": "place_RA", "via": ["place_S"], "to": "place_LB", "action": {"func": "lock", "token": "token_b", "waiting_route": {}}, "destination": "place_Hdash"},
      {"from": "place_LB", "via": ["place_T"], "to": "place_RB", "action": {"func": "release", "token": "token_b", "waiting_route": {}}, "destination": "place_Hdash"},
      {"from": "place_RB", "via": [], "to": "place_LC", "action": {"func": "lock", "token": "token_c", "waiting_route": {"via":[],"to":"place_U"}}, "destination": "place_Hdash"},
      {"from": "place_LC", "via": ["place_H"], "to": "place_Hdash", "action": {"func": "", "token": "", "waiting_route": {}}, "destination": "place_Hdash"},
      {"from": "place_Hdash", "via": ["place_H"], "to": "place_RC", "action": {"func": "release", "token": "token_c", "waiting_route": {}}, "destination": "place_Q"},
      {"from": "place_RC", "via": ["place_F", "place_E", "place_B"], "to": "place_Q", "action": {"func": "", "token": "", "waiting_route": {}}, "destination": "place_Q"}
    ]
  },
  {
    "id": "route_plan_01_02",
    "robot_id": "delivery_robot_02",
    "source": "place_V", "via": "place_L", "destination": "place_Hdash", 
    "routes": [
      {"from": "place_V", "via": [], "to": "place_LA", "action": {"func": "lock", "token":"token_a", "waiting_route": {}}, "destination": "place_L"},
      {"from": "place_LA", "via": ["place_R"], "to": "place_L", "action": {"func": "", "token": "", "waiting_route": {}}, "destination": "place_L"},
      {"from": "place_L", "via": ["place_R"], "to": "place_RA", "action": {"func": "release", "token": "token_a", "waiting_route": {}}, "destination": "place_Hdash"},
      {"from": "place_RA", "via": ["place_S"], "to": "place_LB", "action": {"func": "lock", "token": "token_b", "waiting_route": {}}, "destination": "place_Hdash"},
      {"from": "place_LB", "via": ["place_T"], "to": "place_RB", "action": {"func": "release", "token": "token_b", "waiting_route": {}}, "destination": "place_Hdash"},
      {"from": "place_RB", "via": [], "to": "place_LC", "action": {"func": "lock", "token": "token_c", "waiting_route": {"via":[],"to":"place_U"}}, "destination": "place_Hdash"},
      {"from": "place_LC", "via": ["place_H"], "to": "place_Hdash", "action": {"func": "", "token": "", "waiting_route": {}}, "destination": "place_Hdash"},
      {"from": "place_Hdash", "via": ["place_H"], "to": "place_RC", "action": {"func": "release", "token": "token_c", "waiting_route": {}}, "destination": "place_V"},
      {"from": "place_RC", "via": ["place_F", "place_E", "place_B"], "to": "place_V", "action": {"func": "", "token": "", "waiting_route": {}}, "destination": "place_V"}
    ]
  },
  {
    "id": "route_plan_02_01",
    "robot_id": "delivery_robot_01",
    "source": "place_Q", "via": "place_L", "destination": "place_Idash", 
    "routes": [
      {"from": "place_Q", "via": [], "to": "place_LA", "action": {"func": "lock", "token":"token_a", "waiting_route": {}}, "destination": "place_L"},
      {"from": "place_LA", "via": ["place_R"], "to": "place_L", "action": {"func": "", "token": "", "waiting_route": {}}, "destination": "place_L"},
      {"from": "place_L", "via": ["place_R"], "to": "place_RA", "action": {"func": "release", "token": "token_a", "waiting_route": {}}, "destination": "place_Idash"},
      {"from": "place_RA", "via": ["place_S"], "to": "place_LB", "action": {"func": "lock", "token": "token_b", "waiting_route": {}}, "destination": "place_Idash"},
      {"from": "place_LB", "via": ["place_T"], "to": "place_RB", "action": {"func": "release", "token": "token_b", "waiting_route": {}}, "destination": "place_Idash"},
      {"from": "place_RB", "via": [], "to": "place_LC", "action": {"func": "lock", "token": "token_c", "waiting_route": {"via":[],"to":"place_U"}}, "destination": "place_Idash"},
      {"from": "place_LC", "via": ["place_H", "place_I"], "to": "place_Idash", "action": {"func": "", "token": "", "waiting_route": {}}, "destination": "place_Idash"},
      {"from": "place_Idash", "via": ["place_I", "place_H"], "to": "place_RC", "action": {"func": "release", "token": "token_c", "waiting_route": {}}, "destination": "place_Q"},
      {"from": "place_RC", "via": ["place_F", "place_E", "place_B"], "to": "place_Q", "action": {"func": "", "token": "", "waiting_route": {}}, "destination": "place_Q"}
    ]
  },
  {
    "id": "route_plan_02_02",
    "robot_id": "delivery_robot_02",
    "source": "place_V", "via": "place_L", "destination": "place_Idash", 
    "routes": [
      {"from": "place_V", "via": [], "to": "place_LA", "action": {"func": "lock", "token":"token_a", "waiting_route": {}}, "destination": "place_L"},
      {"from": "place_LA", "via": ["place_R"], "to": "place_L", "action": {"func": "", "token": "", "waiting_route": {}}, "destination": "place_L"},
      {"from": "place_L", "via": ["place_R"], "to": "place_RA", "action": {"func": "release", "token": "token_a", "waiting_route": {}}, "destination": "place_Idash"},
      {"from": "place_RA", "via": ["place_S"], "to": "place_LB", "action": {"func": "lock", "token": "token_b", "waiting_route": {}}, "destination": "place_Idash"},
      {"from": "place_LB", "via": ["place_T"], "to": "place_RB", "action": {"func": "release", "token": "token_b", "waiting_route": {}}, "destination": "place_Idash"},
      {"from": "place_RB", "via": [], "to": "place_LC", "action": {"func": "lock", "token": "token_c", "waiting_route": {"via":[],"to":"place_U"}}, "destination": "place_Idash"},
      {"from": "place_LC", "via": ["place_H", "place_I"], "to": "place_Idash", "action": {"func": "", "token": "", "waiting_route": {}}, "destination": "place_Idash"},
      {"from": "place_Idash", "via": ["place_I", "place_H"], "to": "place_RC", "action": {"func": "release", "token": "token_c", "waiting_route": {}}, "destination": "place_V"},
      {"from": "place_RC", "via": ["place_F", "place_E", "place_B"], "to": "place_V", "action": {"func": "", "token": "", "waiting_route": {}}, "destination": "place_V"}
    ]
  },
  {
    "id": "route_plan_03_01",
    "robot_id": "delivery_robot_01",
    "source": "place_Q", "via": "place_L", "destination": "place_Jdash", 
    "routes": [
      {"from": "place_Q", "via": [], "to": "place_LA", "action": {"func": "lock", "token":"token_a", "waiting_route": {}}, "destination": "place_L"},
      {"from": "place_LA", "via": ["place_R"], "to": "place_L", "action": {"func": "", "token": "", "waiting_route": {}}, "destination": "place_L"},
      {"from": "place_L", "via": ["place_R"], "to": "place_RA", "action": {"func": "release", "token": "token_a", "waiting_route": {}}, "destination": "place_Jdash"},
      {"from": "place_RA", "via": ["place_S"], "to": "place_LB", "action": {"func": "lock", "token": "token_b", "waiting_route": {}}, "destination": "place_Jdash"},
      {"from": "place_LB", "via": ["place_T"], "to": "place_RB", "action": {"func": "release", "token": "token_b", "waiting_route": {}}, "destination": "place_Jdash"},
      {"from": "place_RB", "via": [], "to": "place_LC", "action": {"func": "lock", "token": "token_c", "waiting_route": {"via":[],"to":"place_U"}}, "destination": "place_Jdash"},
      {"from": "place_LC", "via": ["place_H", "place_I", "place_J"], "to": "place_Jdash", "action": {"func": "", "token": "", "waiting_route": {}}, "destination": "place_Jdash"},
      {"from": "place_Jdash", "via": ["place_J", "place_I", "place_H"], "to": "place_RC", "action": {"func": "release", "token": "token_c", "waiting_route": {}}, "destination": "place_Q"},
      {"from": "place_RC", "via": ["place_F", "place_E", "place_B"], "to": "place_Q", "action": {"func": "", "token": "", "waiting_route": {}}, "destination": "place_Q"}
    ]
  },
  {
    "id": "route_plan_03_02",
    "robot_id": "delivery_robot_02",
    "source": "place_V", "via": "place_L", "destination": "place_Jdash", 
    "routes": [
      {"from": "place_V", "via": [], "to": "place_LA", "action": {"func": "lock", "token":"token_a", "waiting_route": {}}, "destination": "place_L"},
      {"from": "place_LA", "via": ["place_R"], "to": "place_L", "action": {"func": "", "token": "", "waiting_route": {}}, "destination": "place_L"},
      {"from": "place_L", "via": ["place_R"], "to": "place_RA", "action": {"func": "release", "token": "token_a", "waiting_route": {}}, "destination": "place_Jdash"},
      {"from": "place_RA", "via": ["place_S"], "to": "place_LB", "action": {"func": "lock", "token": "token_b", "waiting_route": {}}, "destination": "place_Jdash"},
      {"from": "place_LB", "via": ["place_T"], "to": "place_RB", "action": {"func": "release", "token": "token_b", "waiting_route": {}}, "destination": "place_Jdash"},
      {"from": "place_RB", "via": [], "to": "place_LC", "action": {"func": "lock", "token": "token_c", "waiting_route": {"via":[],"to":"place_U"}}, "destination": "place_Jdash"},
      {"from": "place_LC", "via": ["place_H", "place_I", "place_J"], "to": "place_Jdash", "action": {"func": "", "token": "", "waiting_route": {}}, "destination": "place_Jdash"},
      {"from": "place_Jdash", "via": ["place_J", "place_I", "place_H"], "to": "place_RC", "action": {"func": "release", "token": "token_c", "waiting_route": {}}, "destination": "place_V"},
      {"from": "place_RC", "via": ["place_F", "place_E", "place_B"], "to": "place_V", "action": {"func": "", "token": "", "waiting_route": {}}, "destination": "place_V"}
    ]
  },
  {
    "id": "route_plan_04_01",
    "robot_id": "delivery_robot_01",
    "source": "place_Q", "via": "place_G", "destination": "place_Hdash", 
    "routes": [
      {"from": "place_Q", "via": [], "to": "place_LA", "action": {"func": "lock", "token":"token_a", "waiting_route": {}}, "destination": "place_G"},
      {"from": "place_LA", "via": ["place_R"], "to": "place_RA", "action": {"func": "release", "token": "token_a", "waiting_route": {}}, "destination": "place_G"},
      {"from": "place_RA", "via": ["place_S"], "to": "place_LB", "action": {"func": "lock", "token": "token_b", "waiting_route": {}}, "destination": "place_G"},
      {"from": "place_LB", "via": ["place_T"], "to": "place_G", "action": {"func": "", "token": "", "waiting_route": {}}, "destination": "place_G"},
      {"from": "place_G", "via": ["place_T"], "to": "place_RB", "action": {"func": "release", "token": "token_b", "waiting_route": {}}, "destination": "place_Hdash"},
      {"from": "place_RB", "via": [], "to": "place_LC", "action": {"func": "lock", "token": "token_c", "waiting_route": {"via":[],"to":"place_U"}}, "destination": "place_Hdash"},
      {"from": "place_LC", "via": ["place_H"], "to": "place_Hdash", "action": {"func": "", "token": "", "waiting_route": {}}, "destination": "place_Hdash"},
      {"from": "place_Hdash", "via": ["place_H"], "to": "place_RC", "action": {"func": "release", "token": "token_c", "waiting_route": {}}, "destination": "place_Q"},
      {"from": "place_RC", "via": ["place_F", "place_E", "place_B"], "to": "place_Q", "action": {"func": "", "token": "", "waiting_route": {}}, "destination": "place_Q"}
    ]
  },
  {
    "id": "route_plan_04_02",
    "robot_id": "delivery_robot_02",
    "source": "place_V", "via": "place_G", "destination": "place_Hdash", 
    "routes": [
      {"from": "place_V", "via": [], "to": "place_LA", "action": {"func": "lock", "token":"token_a", "waiting_route": {}}, "destination": "place_G"},
      {"from": "place_LA", "via": ["place_R"], "to": "place_RA", "action": {"func": "release", "token": "token_a", "waiting_route": {}}, "destination": "place_G"},
      {"from": "place_RA", "via": ["place_S"], "to": "place_LB", "action": {"func": "lock", "token": "token_b", "waiting_route": {}}, "destination": "place_G"},
      {"from": "place_LB", "via": ["place_T"], "to": "place_G", "action": {"func": "", "token": "", "waiting_route": {}}, "destination": "place_G"},
      {"from": "place_G", "via": ["place_T"], "to": "place_RB", "action": {"func": "release", "token": "token_b", "waiting_route": {}}, "destination": "place_Hdash"},
      {"from": "place_RB", "via": [], "to": "place_LC", "action": {"func": "lock", "token": "token_c", "waiting_route": {"via":[],"to":"place_U"}}, "destination": "place_Hdash"},
      {"from": "place_LC", "via": ["place_H"], "to": "place_Hdash", "action": {"func": "", "token": "", "waiting_route": {}}, "destination": "place_Hdash"},
      {"from": "place_Hdash", "via": ["place_H"], "to": "place_RC", "action": {"func": "release", "token": "token_c", "waiting_route": {}}, "destination": "place_V"},
      {"from": "place_RC", "via": ["place_F", "place_E", "place_B"], "to": "place_V", "action": {"func": "", "token": "", "waiting_route": {}}, "destination": "place_V"}
    ]
  },
  {
    "id": "route_plan_05_01",
    "robot_id": "delivery_robot_01",
    "source": "place_Q", "via": "place_G", "destination": "place_Idash", 
    "routes": [
      {"from": "place_Q", "via": [], "to": "place_LA", "action": {"func": "lock", "token":"token_a", "waiting_route": {}}, "destination": "place_G"},
      {"from": "place_LA", "via": ["place_R"], "to": "place_RA", "action": {"func": "release", "token": "token_a", "waiting_route": {}}, "destination": "place_G"},
      {"from": "place_RA", "via": ["place_S"], "to": "place_LB", "action": {"func": "lock", "token": "token_b", "waiting_route": {}}, "destination": "place_G"},
      {"from": "place_LB", "via": ["place_T"], "to": "place_G", "action": {"func": "", "token": "", "waiting_route": {}}, "destination": "place_G"},
      {"from": "place_G", "via": ["place_T"], "to": "place_RB", "action": {"func": "release", "token": "token_b", "waiting_route": {}}, "destination": "place_Idash"},
      {"from": "place_RB", "via": [], "to": "place_LC", "action": {"func": "lock", "token": "token_c", "waiting_route": {"via":[],"to":"place_U"}}, "destination": "place_Idash"},
      {"from": "place_LC", "via": ["place_H", "place_I"], "to": "place_Idash", "action": {"func": "", "token": "", "waiting_route": {}}, "destination": "place_Idash"},
      {"from": "place_Idash", "via": ["place_I", "place_H"], "to": "place_RC", "action": {"func": "release", "token": "token_c", "waiting_route": {}}, "destination": "place_Q"},
      {"from": "place_RC", "via": ["place_F", "place_E", "place_B"], "to": "place_Q", "action": {"func": "", "token": "", "waiting_route": {}}, "destination": "place_Q"}
    ]
  },
  {
    "id": "route_plan_05_02",
    "robot_id": "delivery_robot_02",
    "source": "place_V", "via": "place_G", "destination": "place_Idash", 
    "routes": [
      {"from": "place_V", "via": [], "to": "place_LA", "action": {"func": "lock", "token":"token_a", "waiting_route": {}}, "destination": "place_G"},
      {"from": "place_LA", "via": ["place_R"], "to": "place_RA", "action": {"func": "release", "token": "token_a", "waiting_route": {}}, "destination": "place_G"},
      {"from": "place_RA", "via": ["place_S"], "to": "place_LB", "action": {"func": "lock", "token": "token_b", "waiting_route": {}}, "destination": "place_G"},
      {"from": "place_LB", "via": ["place_T"], "to": "place_G", "action": {"func": "", "token": "", "waiting_route": {}}, "destination": "place_G"},
      {"from": "place_G", "via": ["place_T"], "to": "place_RB", "action": {"func": "release", "token": "token_b", "waiting_route": {}}, "destination": "place_Idash"},
      {"from": "place_RB", "via": [], "to": "place_LC", "action": {"func": "lock", "token": "token_c", "waiting_route": {"via":[],"to":"place_U"}}, "destination": "place_Idash"},
      {"from": "place_LC", "via": ["place_H", "place_I"], "to": "place_Idash", "action": {"func": "", "token": "", "waiting_route": {}}, "destination": "place_Idash"},
      {"from": "place_Idash", "via": ["place_I", "place_H"], "to": "place_RC", "action": {"func": "release", "token": "token_c", "waiting_route": {}}, "destination": "place_V"},
      {"from": "place_RC", "via": ["place_F", "place_E", "place_B"], "to": "place_V", "action": {"func": "", "token": "", "waiting_route": {}}, "destination": "place_V"}
    ]
  },
  {
    "id": "route_plan_06_01",
    "robot_id": "delivery_robot_01",
    "source": "place_Q", "via": "place_G", "destination": "place_Jdash", 
    "routes": [
      {"from": "place_Q", "via": [], "to": "place_LA", "action": {"func": "lock", "token":"token_a", "waiting_route": {}}, "destination": "place_G"},
      {"from": "place_LA", "via": ["place_R"], "to": "place_RA", "action": {"func": "release", "token": "token_a", "waiting_route": {}}, "destination": "place_G"},
      {"from": "place_RA", "via": ["place_S"], "to": "place_LB", "action": {"func": "lock", "token": "token_b", "waiting_route": {}}, "destination": "place_G"},
      {"from": "place_LB", "via": ["place_T"], "to": "place_G", "action": {"func": "", "token": "", "waiting_route": {}}, "destination": "place_G"},
      {"from": "place_G", "via": ["place_T"], "to": "place_RB", "action": {"func": "release", "token": "token_b", "waiting_route": {}}, "destination": "place_Jdash"},
      {"from": "place_RB", "via": [], "to": "place_LC", "action": {"func": "lock", "token": "token_c", "waiting_route": {"via":[],"to":"place_U"}}, "destination": "place_Jdash"},
      {"from": "place_LC", "via": ["place_H", "place_I", "place_J"], "to": "place_Jdash", "action": {"func": "", "token": "", "waiting_route": {}}, "destination": "place_Jdash"},
      {"from": "place_Jdash", "via": ["place_J", "place_I", "place_H"], "to": "place_RC", "action": {"func": "release", "token": "token_c", "waiting_route": {}}, "destination": "place_Q"},
      {"from": "place_RC", "via": ["place_F", "place_E", "place_B"], "to": "place_Q", "action": {"func": "", "token": "", "waiting_route": {}}, "destination": "place_Q"}
    ]
  },
  {
    "id": "route_plan_06_02",
    "robot_id": "delivery_robot_02",
    "source": "place_V", "via": "place_G", "destination": "place_Jdash", 
    "routes": [
      {"from": "place_V", "via": [], "to": "place_LA", "action": {"func": "lock", "token":"token_a", "waiting_route": {}}, "destination": "place_G"},
      {"from": "place_LA", "via": ["place_R"], "to": "place_RA", "action": {"func": "release", "token": "token_a", "waiting_route": {}}, "destination": "place_G"},
      {"from": "place_RA", "via": ["place_S"], "to": "place_LB", "action": {"func": "lock", "token": "token_b", "waiting_route": {}}, "destination": "place_G"},
      {"from": "place_LB", "via": ["place_T"], "to": "place_G", "action": {"func": "", "token": "", "waiting_route": {}}, "destination": "place_G"},
      {"from": "place_G", "via": ["place_T"], "to": "place_RB", "action": {"func": "release", "token": "token_b", "waiting_route": {}}, "destination": "place_Jdash"},
      {"from": "place_RB", "via": [], "to": "place_LC", "action": {"func": "lock", "token": "token_c", "waiting_route": {"via":[],"to":"place_U"}}, "destination": "place_Jdash"},
      {"from": "place_LC", "via": ["place_H", "place_I", "place_J"], "to": "place_Jdash", "action": {"func": "", "token": "", "waiting_route": {}}, "destination": "place_Jdash"},
      {"from": "place_Jdash", "via": ["place_J", "place_I", "place_H"], "to": "place_RC", "action": {"func": "release", "token": "token_c", "waiting_route": {}}, "destination": "place_V"},
      {"from": "place_RC", "via": ["place_F", "place_E", "place_B"], "to": "place_V", "action": {"func": "", "token": "", "waiting_route": {}}, "destination": "place_V"}
    ]
  },
  {
    "id": "route_plan_07_01",
    "robot_id": "delivery_robot_01",
    "source": "place_Q", "via": "place_G|place_L", "destination": "place_Hdash", 
    "routes": [
      {"from": "place_Q", "via": [], "to": "place_LA", "action": {"func": "lock", "token":"token_a", "waiting_route": {}}, "destination": "place_L"},
      {"from": "place_LA", "via": ["place_R"], "to": "place_L", "action": {"func": "", "token": "", "waiting_route": {}}, "destination": "place_L"},
      {"from": "place_L", "via": ["place_R"], "to": "place_RA", "action": {"func": "release", "token": "token_a", "waiting_route": {}}, "destination": "place_G"},
      {"from": "place_RA", "via": ["place_S"], "to": "place_LB", "action": {"func": "lock", "token": "token_b", "waiting_route": {}}, "destination": "place_G"},
      {"from": "place_LB", "via": ["place_T"], "to": "place_G", "action": {"func": "", "token": "", "waiting_route": {}}, "destination": "place_G"},
      {"from": "place_G", "via": ["place_T"], "to": "place_RB", "action": {"func": "release", "token": "token_b", "waiting_route": {}}, "destination": "place_Hdash"},
      {"from": "place_RB", "via": [], "to": "place_LC", "action": {"func": "lock", "token": "token_c", "waiting_route": {"via":[],"to":"place_U"}}, "destination": "place_Hdash"},
      {"from": "place_LC", "via": ["place_H"], "to": "place_Hdash", "action": {"func": "", "token": "", "waiting_route": {}}, "destination": "place_Hdash"},
      {"from": "place_Hdash", "via": ["place_H"], "to": "place_RC", "action": {"func": "release", "token": "token_c", "waiting_route": {}}, "destination": "place_Q"},
      {"from": "place_RC", "via": ["place_F", "place_E", "place_B"], "to": "place_Q", "action": {"func": "", "token": "", "waiting_route": {}}, "destination": "place_Q"}
    ]
  },
  {
    "id": "route_plan_07_02",
    "robot_id": "delivery_robot_02",
    "source": "place_V", "via": "place_G|place_L", "destination": "place_Hdash", 
    "routes": [
      {"from": "place_V", "via": [], "to": "place_LA", "action": {"func": "lock", "token":"token_a", "waiting_route": {}}, "destination": "place_L"},
      {"from": "place_LA", "via": ["place_R"], "to": "place_L", "action": {"func": "", "token": "", "waiting_route": {}}, "destination": "place_L"},
      {"from": "place_L", "via": ["place_R"], "to": "place_RA", "action": {"func": "release", "token": "token_a", "waiting_route": {}}, "destination": "place_G"},
      {"from": "place_RA", "via": ["place_S"], "to": "place_LB", "action": {"func": "lock", "token": "token_b", "waiting_route": {}}, "destination": "place_G"},
      {"from": "place_LB", "via": ["place_T"], "to": "place_G", "action": {"func": "", "token": "", "waiting_route": {}}, "destination": "place_G"},
      {"from": "place_G", "via": ["place_T"], "to": "place_RB", "action": {"func": "release", "token": "token_b", "waiting_route": {}}, "destination": "place_Hdash"},
      {"from": "place_RB", "via": [], "to": "place_LC", "action": {"func": "lock", "token": "token_c", "waiting_route": {"via":[],"to":"place_U"}}, "destination": "place_Hdash"},
      {"from": "place_LC", "via": ["place_H"], "to": "place_Hdash", "action": {"func": "", "token": "", "waiting_route": {}}, "destination": "place_Hdash"},
      {"from": "place_Hdash", "via": ["place_H"], "to": "place_RC", "action": {"func": "release", "token": "token_c", "waiting_route": {}}, "destination": "place_V"},
      {"from": "place_RC", "via": ["place_F", "place_E", "place_B"], "to": "place_V", "action": {"func": "", "token": "", "waiting_route": {}}, "destination": "place_V"}
    ]
  },
  {
    "id": "route_plan_08_01",
    "robot_id": "delivery_robot_01",
    "source": "place_Q", "via": "place_G|place_L", "destination": "place_Idash", 
    "routes": [
      {"from": "place_Q", "via": [], "to": "place_LA", "action": {"func": "lock", "token":"token_a", "waiting_route": {}}, "destination": "place_L"},
      {"from": "place_LA", "via": ["place_R"], "to": "place_L", "action": {"func": "", "token": "", "waiting_route": {}}, "destination": "place_L"},
      {"from": "place_L", "via": ["place_R"], "to": "place_RA", "action": {"func": "release", "token": "token_a", "waiting_route": {}}, "destination": "place_G"},
      {"from": "place_RA", "via": ["place_S"], "to": "place_LB", "action": {"func": "lock", "token": "token_b", "waiting_route": {}}, "destination": "place_G"},
      {"from": "place_LB", "via": ["place_T"], "to": "place_G", "action": {"func": "", "token": "", "waiting_route": {}}, "destination": "place_G"},
      {"from": "place_G", "via": ["place_T"], "to": "place_RB", "action": {"func": "release", "token": "token_b", "waiting_route": {}}, "destination": "place_Idash"},
      {"from": "place_RB", "via": [], "to": "place_LC", "action": {"func": "lock", "token": "token_c", "waiting_route": {"via":[],"to":"place_U"}}, "destination": "place_Idash"},
      {"from": "place_LC", "via": ["place_H", "place_I"], "to": "place_Idash", "action": {"func": "", "token": "", "waiting_route": {}}, "destination": "place_Idash"},
      {"from": "place_Idash", "via": ["place_I", "place_H"], "to": "place_RC", "action": {"func": "release", "token": "token_c", "waiting_route": {}}, "destination": "place_Q"},
      {"from": "place_RC", "via": ["place_F", "place_E", "place_B"], "to": "place_Q", "action": {"func": "", "token": "", "waiting_route": {}}, "destination": "place_Q"}
    ]
  },
  {
    "id": "route_plan_08_02",
    "robot_id": "delivery_robot_02",
    "source": "place_V", "via": "place_G|place_L", "destination": "place_Idash", 
    "routes": [
      {"from": "place_V", "via": [], "to": "place_LA", "action": {"func": "lock", "token":"token_a", "waiting_route": {}}, "destination": "place_L"},
      {"from": "place_LA", "via": ["place_R"], "to": "place_L", "action": {"func": "", "token": "", "waiting_route": {}}, "destination": "place_L"},
      {"from": "place_L", "via": ["place_R"], "to": "place_RA", "action": {"func": "release", "token": "token_a", "waiting_route": {}}, "destination": "place_G"},
      {"from": "place_RA", "via": ["place_S"], "to": "place_LB", "action": {"func": "lock", "token": "token_b", "waiting_route": {}}, "destination": "place_G"},
      {"from": "place_LB", "via": ["place_T"], "to": "place_G", "action": {"func": "", "token": "", "waiting_route": {}}, "destination": "place_G"},
      {"from": "place_G", "via": ["place_T"], "to": "place_RB", "action": {"func": "release", "token": "token_b", "waiting_route": {}}, "destination": "place_Idash"},
      {"from": "place_RB", "via": [], "to": "place_LC", "action": {"func": "lock", "token": "token_c", "waiting_route": {"via":[],"to":"place_U"}}, "destination": "place_Idash"},
      {"from": "place_LC", "via": ["place_H", "place_I"], "to": "place_Idash", "action": {"func": "", "token": "", "waiting_route": {}}, "destination": "place_Idash"},
      {"from": "place_Idash", "via": ["place_I", "place_H"], "to": "place_RC", "action": {"func": "release", "token": "token_c", "waiting_route": {}}, "destination": "place_V"},
      {"from": "place_RC", "via": ["place_F", "place_E", "place_B"], "to": "place_V", "action": {"func": "", "token": "", "waiting_route": {}}, "destination": "place_V"}
    ]
  },
  {
    "id": "route_plan_09_01",
    "robot_id": "delivery_robot_01",
    "source": "place_Q", "via": "place_G|place_L", "destination": "place_Jdash", 
    "routes": [
      {"from": "place_Q", "via": [], "to": "place_LA", "action": {"func": "lock", "token":"token_a", "waiting_route": {}}, "destination": "place_L"},
      {"from": "place_LA", "via": ["place_R"], "to": "place_L", "action": {"func": "", "token": "", "waiting_route": {}}, "destination": "place_L"},
      {"from": "place_L", "via": ["place_R"], "to": "place_RA", "action": {"func": "release", "token": "token_a", "waiting_route": {}}, "destination": "place_G"},
      {"from": "place_RA", "via": ["place_S"], "to": "place_LB", "action": {"func": "lock", "token": "token_b", "waiting_route": {}}, "destination": "place_G"},
      {"from": "place_LB", "via": ["place_T"], "to": "place_G", "action": {"func": "", "token": "", "waiting_route": {}}, "destination": "place_G"},
      {"from": "place_G", "via": ["place_T"], "to": "place_RB", "action": {"func": "release", "token": "token_b", "waiting_route": {}}, "destination": "place_Jdash"},
      {"from": "place_RB", "via": [], "to": "place_LC", "action": {"func": "lock", "token": "token_c", "waiting_route": {"via":[],"to":"place_U"}}, "destination": "place_Jdash"},
      {"from": "place_LC", "via": ["place_H", "place_I","place_J"], "to": "place_Jdash", "action": {"func": "", "token": "", "waiting_route": {}}, "destination": "place_Jdash"},
      {"from": "place_Jdash", "via": ["place_J", "place_I", "place_H"], "to": "place_RC", "action": {"func": "release", "token": "token_c", "waiting_route": {}}, "destination": "place_Q"},
      {"from": "place_RC", "via": ["place_F", "place_E", "place_B"], "to": "place_Q", "action": {"func": "", "token": "", "waiting_route": {}}, "destination": "place_Q"}
    ]
  },
  {
    "id": "route_plan_09_02",
    "robot_id": "delivery_robot_02",
    "source": "place_V", "via": "place_G|place_L", "destination": "place_Jdash", 
    "routes": [
      {"from": "place_V", "via": [], "to": "place_LA", "action": {"func": "lock", "token":"token_a", "waiting_route": {}}, "destination": "place_L"},
      {"from": "place_LA", "via": ["place_R"], "to": "place_L", "action": {"func": "", "token": "", "waiting_route": {}}, "destination": "place_L"},
      {"from": "place_L", "via": ["place_R"], "to": "place_RA", "action": {"func": "release", "token": "token_a", "waiting_route": {}}, "destination": "place_G"},
      {"from": "place_RA", "via": ["place_S"], "to": "place_LB", "action": {"func": "lock", "token": "token_b", "waiting_route": {}}, "destination": "place_G"},
      {"from": "place_LB", "via": ["place_T"], "to": "place_G", "action": {"func": "", "token": "", "waiting_route": {}}, "destination": "place_G"},
      {"from": "place_G", "via": ["place_T"], "to": "place_RB", "action": {"func": "release", "token": "token_b", "waiting_route": {}}, "destination": "place_Jdash"},
      {"from": "place_RB", "via": [], "to": "place_LC", "action": {"func": "lock", "token": "token_c", "waiting_route": {"via":[],"to":"place_U"}}, "destination": "place_Jdash"},
      {"from": "place_LC", "via": ["place_H", "place_I","place_J"], "to": "place_Jdash", "action": {"func": "", "token": "", "waiting_route": {}}, "destination": "place_Jdash"},
      {"from": "place_Jdash", "via": ["place_J", "place_I", "place_H"], "to": "place_RC", "action": {"func": "release", "token": "token_c", "waiting_route": {}}, "destination": "place_V"},
      {"from": "place_RC", "via": ["place_F", "place_E", "place_B"], "to": "place_V", "action": {"func": "", "token": "", "waiting_route": {}}, "destination": "place_V"}
    ]
  }
]
__EOD__
)

TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
LEN=$(echo ${ROUTE_PLANS} | jq length)
count=0
if [ ${LEN} -gt 0 ]; then
  for i in $(seq 0 $((${LEN} - 1))); do
    id=$(echo ${ROUTE_PLANS} | jq .[${i}].id)
    robot_id=$(echo ${ROUTE_PLANS} | jq .[${i}].robot_id)
    source=$(echo ${ROUTE_PLANS} | jq .[${i}].source)
    via=$(echo ${ROUTE_PLANS} | jq .[${i}].via)
    destination=$(echo ${ROUTE_PLANS} | jq .[${i}].destination)
    routes=$(echo ${ROUTE_PLANS} | jq .[${i}].routes)
    echo "${id} (robot_id=${robot_id}, source=${source}, via=${via}, destination=${destination}) will be registerd."
    curl -i -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DELIVERY_ROBOT_SERVICEPATH}" -H "Content-Type: application/json" https://api.${DOMAIN}/orion/v2/entities/ -X POST -d @-<<__EOS__
{
  "id": ${id},
  "type": "route_plan",
  "TimeInstant": {
    "type": "ISO8601",
    "value": "$(iso8601)"
  },
  "source": {
    "type": "string",
    "value": ${source}
  },
  "destination": {
    "type": "string",
    "value": ${destination}
  },
  "via": {
    "type": "string",
    "value": ${via}
  },
  "routes": {
    "type": "array",
    "value": ${routes}
  },
  "robot_id": {
    "type": "string",
    "value": ${robot_id}
  }
}
__EOS__
  let ++count
  done
fi
echo "${count} entities were registered."

example)

```
"route_plan_01_01" (robot_id="delivery_robot_01", source="place_Q", via="place_L", destination="place_Hdash") will be registerd.
HTTP/1.1 100 Continue

HTTP/1.1 201 Created
content-length: 0
location: /v2/entities/route_plan_01_01?type=route_plan
fiware-correlator: 0d6c4226-01e5-11ea-bfd1-2621012d3c7f
date: Fri, 08 Nov 2019 05:03:07 GMT
x-envoy-upstream-service-time: 11
server: envoy

"route_plan_01_02" (robot_id="delivery_robot_02", source="place_V", via="place_L", destination="place_Hdash") will be registerd.
HTTP/1.1 100 Continue

HTTP/1.1 201 Created
content-length: 0
location: /v2/entities/route_plan_01_02?type=route_plan
fiware-correlator: 0dc80e9e-01e5-11ea-acd2-aaa36e462053
date: Fri, 08 Nov 2019 05:03:07 GMT
x-envoy-upstream-service-time: 7
server: envoy

...
"route_plan_09_02" (robot_id="delivery_robot_02", source="place_V", via="place_G|place_L", destination="place_Jdash") will be registerd.
HTTP/1.1 100 Continue

HTTP/1.1 201 Created
content-length: 0
location: /v2/entities/route_plan_09_02?type=route_plan
fiware-correlator: 1241d130-01e5-11ea-af0c-aaa36e462053
date: Fri, 08 Nov 2019 05:03:15 GMT
x-envoy-upstream-service-time: 6
server: envoy

18 entities were registered.
```

### confirm registered `route_plan` entities

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DELIVERY_ROBOT_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/?type=route_plan | jq .

example)
```json
[
  {
    "id": "route_plan_01_01",
    "type": "route_plan",
    "TimeInstant": {
      "type": "ISO8601",
      "value": "2019-11-08T05:03:07.00Z",
      "metadata": {}
    },
    "destination": {
      "type": "string",
      "value": "place_Hdash",
      "metadata": {}
    },
    "robot_id": {
      "type": "string",
      "value": "delivery_robot_01",
      "metadata": {}
    },
    "routes": {
      "type": "array",
      "value": [
        {
          "from": "place_Q",
          "via": [],
          "to": "place_LA",
          "action": {
            "func": "lock",
            "token": "token_a",
            "waiting_route": {}
          },
          "destination": "place_L"
        },
        {
          "from": "place_LA",
          "via": [
            "place_R"
          ],
          "to": "place_L",
          "action": {
            "func": "",
            "token": "",
            "waiting_route": {}
          },
          "destination": "place_L"
        },
        {
          "from": "place_L",
          "via": [
            "place_R"
          ],
          "to": "place_RA",
          "action": {
            "func": "release",
            "token": "token_a",
            "waiting_route": {}
          },
          "destination": "place_Hdash"
        },
        {
          "from": "place_RA",
          "via": [
            "place_S"
          ],
          "to": "place_LB",
          "action": {
            "func": "lock",
            "token": "token_b",
            "waiting_route": {}
          },
          "destination": "place_Hdash"
        },
        {
          "from": "place_LB",
          "via": [
            "place_T"
          ],
          "to": "place_RB",
          "action": {
            "func": "release",
            "token": "token_b",
            "waiting_route": {}
          },
          "destination": "place_Hdash"
        },
        {
          "from": "place_RB",
          "via": [],
          "to": "place_LC",
          "action": {
            "func": "lock",
            "token": "token_c",
            "waiting_route": {
              "via": [],
              "to": "place_U"
            }
          },
          "destination": "place_Hdash"
        },
        {
          "from": "place_LC",
          "via": [
            "place_H"
          ],
          "to": "place_Hdash",
          "action": {
            "func": "",
            "token": "",
            "waiting_route": {}
          },
          "destination": "place_Hdash"
        },
        {
          "from": "place_Hdash",
          "via": [
            "place_H"
          ],
          "to": "place_RC",
          "action": {
            "func": "release",
            "token": "token_c",
            "waiting_route": {}
          },
          "destination": "place_Q"
        },
        {
          "from": "place_RC",
          "via": [
            "place_F",
            "place_E",
            "place_B"
          ],
          "to": "place_Q",
          "action": {
            "func": "",
            "token": "",
            "waiting_route": {}
          },
          "destination": "place_Q"
        }
      ],
      "metadata": {}
    },
    "source": {
      "type": "string",
      "value": "place_Q",
      "metadata": {}
    },
    "via": {
      "type": "string",
      "value": "place_L",
      "metadata": {}
    }
  },

...

  {
    "id": "route_plan_09_02",
    "type": "route_plan",
    "TimeInstant": {
      "type": "ISO8601",
      "value": "2019-11-08T05:03:15.00Z",
      "metadata": {}
    },
    "destination": {
      "type": "string",
      "value": "place_Jdash",
      "metadata": {}
    },
    "robot_id": {
      "type": "string",
      "value": "delivery_robot_02",
      "metadata": {}
    },
    "routes": {
      "type": "array",
      "value": [
        {
          "from": "place_V",
          "via": [],
          "to": "place_LA",
          "action": {
            "func": "lock",
            "token": "token_a",
            "waiting_route": {}
          },
          "destination": "place_L"
        },
        {
          "from": "place_LA",
          "via": [
            "place_R"
          ],
          "to": "place_L",
          "action": {
            "func": "",
            "token": "",
            "waiting_route": {}
          },
          "destination": "place_L"
        },
        {
          "from": "place_L",
          "via": [
            "place_R"
          ],
          "to": "place_RA",
          "action": {
            "func": "release",
            "token": "token_a",
            "waiting_route": {}
          },
          "destination": "place_G"
        },
        {
          "from": "place_RA",
          "via": [
            "place_S"
          ],
          "to": "place_LB",
          "action": {
            "func": "lock",
            "token": "token_b",
            "waiting_route": {}
          },
          "destination": "place_G"
        },
        {
          "from": "place_LB",
          "via": [
            "place_T"
          ],
          "to": "place_G",
          "action": {
            "func": "",
            "token": "",
            "waiting_route": {}
          },
          "destination": "place_G"
        },
        {
          "from": "place_G",
          "via": [
            "place_T"
          ],
          "to": "place_RB",
          "action": {
            "func": "release",
            "token": "token_b",
            "waiting_route": {}
          },
          "destination": "place_Jdash"
        },
        {
          "from": "place_RB",
          "via": [],
          "to": "place_LC",
          "action": {
            "func": "lock",
            "token": "token_c",
            "waiting_route": {
              "via": [],
              "to": "place_U"
            }
          },
          "destination": "place_Jdash"
        },
        {
          "from": "place_LC",
          "via": [
            "place_H",
            "place_I",
            "place_J"
          ],
          "to": "place_Jdash",
          "action": {
            "func": "",
            "token": "",
            "waiting_route": {}
          },
          "destination": "place_Jdash"
        },
        {
          "from": "place_Jdash",
          "via": [
            "place_J",
            "place_I",
            "place_H"
          ],
          "to": "place_RC",
          "action": {
            "func": "release",
            "token": "token_c",
            "waiting_route": {}
          },
          "destination": "place_V"
        },
        {
          "from": "place_RC",
          "via": [
            "place_F",
            "place_E",
            "place_B"
          ],
          "to": "place_V",
          "action": {
            "func": "",
            "token": "",
            "waiting_route": {}
          },
          "destination": "place_V"
        }
      ],
      "metadata": {}
    },
    "source": {
      "type": "string",
      "value": "place_V",
      "metadata": {}
    },
    "via": {
      "type": "string",
      "value": "place_G|place_L",
      "metadata": {}
    }
  }
]
```

### delete existing `token` entities

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
EXISTING_TOKENS=$(curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${TOKEN_SERVICEPATH}" "https://api.${DOMAIN}/orion/v2/entities/?type=${TOKEN_TYPE}&limit=1000&attrs=id")
LEN=$(echo ${EXISTING_TOKENS} | jq length)
count=0
if [ ${LEN} -gt 0 ]; then
  for i in $(seq 0 $((${LEN} - 1))); do
    id=$(echo ${EXISTING_TOKENS} | jq .[${i}].id -r)
    echo "${id} will be deleted."
    curl -i -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${TOKEN_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/${id}/?type=${TOKEN_TYPE} -X DELETE
    let ++count
  done
fi
echo "${count} entities were deleted."

example)
```
token_a will be deleted.
HTTP/1.1 204 No Content
content-length: 0
fiware-correlator: 80e84c0e-01e5-11ea-acd2-aaa36e462053
date: Fri, 08 Nov 2019 05:06:20 GMT
x-envoy-upstream-service-time: 5
server: envoy

...

3 entities were deleted.
```

### register `token` entities

In [ ]:
TOKENS=$(cat << __EOD__
[
  {
    "id": "token_a"
  },
  {
    "id": "token_b"
  },
  {
    "id": "token_c"
  }
]
__EOD__
)

TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
LEN=$(echo ${TOKENS} | jq length)
count=0
if [ ${LEN} -gt 0 ]; then
  for i in $(seq 0 $((${LEN} - 1))); do
    id=$(echo ${TOKENS} | jq .[${i}].id)
    echo "${id} will be registerd."
    curl -i -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${TOKEN_SERVICEPATH}" -H "Content-Type: application/json" https://api.${DOMAIN}/orion/v2/entities/ -X POST -d @-<<__EOS__
{
  "id": ${id},
  "type": "${TOKEN_TYPE}",
  "TimeInstant": {
    "type": "ISO8601",
    "value": "$(iso8601)"
  },
  "is_locked": {
    "type": "boolean",
    "value": false
  },
  "lock_owner_id": {
    "type": "string",
    "value": ""
  },
  "waitings": {
    "type": "array",
    "value": []
  }
}
__EOS__
  let ++count
  done
fi
echo "${count} entities were registered."

example)

```
"token_a" will be registerd.
HTTP/1.1 201 Created
content-length: 0
location: /v2/entities/token_a?type=token
fiware-correlator: 8d8ae296-01e5-11ea-8f75-2621012d3c7f
date: Fri, 08 Nov 2019 05:06:42 GMT
x-envoy-upstream-service-time: 8
server: envoy

...

3 entities were registered.
```

### confirm registered `token` entities

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${TOKEN_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/?type=${TOKEN_TYPE} | jq .

example)
```json
[
  {
    "id": "token_a",
    "type": "token",
    "TimeInstant": {
      "type": "ISO8601",
      "value": "2019-11-08T05:06:42.00Z",
      "metadata": {}
    },
    "is_locked": {
      "type": "boolean",
      "value": false,
      "metadata": {}
    },
    "lock_owner_id": {
      "type": "string",
      "value": "",
      "metadata": {}
    },
    "waitings": {
      "type": "array",
      "value": [],
      "metadata": {}
    }
  },

...

  {
    "id": "token_c",
    "type": "token",
    "TimeInstant": {
      "type": "ISO8601",
      "value": "2019-11-08T05:06:42.00Z",
      "metadata": {}
    },
    "is_locked": {
      "type": "boolean",
      "value": false,
      "metadata": {}
    },
    "lock_owner_id": {
      "type": "string",
      "value": "",
      "metadata": {}
    },
    "waitings": {
      "type": "array",
      "value": [],
      "metadata": {}
    }
  }
]
```

## register `robot-controller` as a subscriber of delivery robots

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -i -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-ServicePath: ${DELIVERY_ROBOT_SERVICEPATH}" -H "Content-Type: application/json" https://api.${DOMAIN}/orion/v2/subscriptions/ -X POST -d @- <<__EOS__
{
  "subject": {
    "entities": [{
      "idPattern": "${DELIVERY_ROBOT_ID_PREFIX}.*",
      "type": "${DELIVERY_ROBOT_TYPE}"
    }],
    "condition": {
      "attrs": ["mode"]
    }
  },
  "notification": {
    "http": {
      "url": "http://robot-controller:3000/api/v1/robots/notifications/"
    },
    "attrs": ["mode"]
  }
}
__EOS__

example)
```
HTTP/1.1 201 Created
content-length: 0
location: /v2/subscriptions/5dc4f813498c0575719a88fd
fiware-correlator: ab17e354-01e5-11ea-bfd1-2621012d3c7f
date: Fri, 08 Nov 2019 05:07:31 GMT
x-envoy-upstream-service-time: 5
server: envoy
```

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-ServicePath: ${DELIVERY_ROBOT_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/subscriptions/ | jq .

example)
```json
[
  {
    "id": "5dca5e7f498c0575719a88ff",
    "status": "active",
    "subject": {
      "entities": [
        {
          "idPattern": "delivery_robot_.*",
          "type": "delivery_robot"
        }
      ],
      "condition": {
        "attrs": [
          "time",
          "mode",
          "errors",
          "pose",
          "destination",
          "covariance",
          "battery"
        ]
      }
    },
    "notification": {
      "timesSent": 9,
      "lastNotification": "2019-11-12T08:28:19.00Z",
      "attrs": [
        "time",
        "mode",
        "errors",
        "pose",
        "destination",
        "covariance",
        "battery"
      ],
      "attrsFormat": "legacy",
      "http": {
        "url": "http://cygnus-mongo:5050/notify"
      },
      "lastSuccess": "2019-11-12T08:28:19.00Z",
      "lastSuccessCode": 200
    }
  },
  {
    "id": "5dca5e84ac3c449033e9431f",
    "status": "active",
    "subject": {
      "entities": [
        {
          "idPattern": "delivery_robot_.*",
          "type": "delivery_robot"
        }
      ],
      "condition": {
        "attrs": [
          "send_cmd",
          "send_cmd_status",
          "send_cmd_info"
        ]
      }
    },
    "notification": {
      "timesSent": 7,
      "lastNotification": "2019-11-12T08:17:12.00Z",
      "attrs": [
        "send_cmd",
        "send_cmd_status",
        "send_cmd_info"
      ],
      "attrsFormat": "legacy",
      "http": {
        "url": "http://cygnus-mongo:5050/notify"
      },
      "lastSuccess": "2019-11-12T08:17:12.00Z",
      "lastSuccessCode": 200
    }
  },
  {
    "id": "5dca5e8b02f380ec4350d51e",
    "status": "active",
    "subject": {
      "entities": [
        {
          "idPattern": "delivery_robot_.*",
          "type": "delivery_robot"
        }
      ],
      "condition": {
        "attrs": [
          "current_routes",
          "navigating_waypoints",
          "remaining_waypoints_list",
          "order"
        ]
      }
    },
    "notification": {
      "timesSent": 11,
      "lastNotification": "2019-11-12T08:28:19.00Z",
      "attrs": [
        "current_routes",
        "navigating_waypoints",
        "remaining_waypoints_list",
        "order"
      ],
      "attrsFormat": "legacy",
      "http": {
        "url": "http://cygnus-mongo:5050/notify"
      },
      "lastSuccess": "2019-11-12T08:28:19.00Z",
      "lastSuccessCode": 200
    }
  },
  {
    "id": "5dca5e90ac3c449033e94320",
    "status": "active",
    "subject": {
      "entities": [
        {
          "idPattern": "delivery_robot_.*",
          "type": "delivery_robot"
        }
      ],
      "condition": {
        "attrs": [
          "send_emg",
          "send_emg_status",
          "send_emg_info"
        ]
      }
    },
    "notification": {
      "timesSent": 3,
      "lastNotification": "2019-11-12T07:36:55.00Z",
      "attrs": [
        "send_emg",
        "send_emg_status",
        "send_emg_info"
      ],
      "attrsFormat": "legacy",
      "http": {
        "url": "http://cygnus-mongo:5050/notify"
      },
      "lastSuccess": "2019-11-12T07:36:55.00Z",
      "lastSuccessCode": 200
    }
  },
  {
    "id": "5dca5ed7498c0575719a8900",
    "status": "active",
    "subject": {
      "entities": [
        {
          "idPattern": "delivery_robot_.*",
          "type": "delivery_robot"
        }
      ],
      "condition": {
        "attrs": [
          "current_mode",
          "current_state"
        ]
      }
    },
    "notification": {
      "timesSent": 8,
      "lastNotification": "2019-11-12T08:28:19.00Z",
      "attrs": [
        "current_mode",
        "current_state"
      ],
      "attrsFormat": "legacy",
      "http": {
        "url": "http://cygnus-mongo:5050/notify"
      },
      "lastSuccess": "2019-11-12T08:28:19.00Z",
      "lastSuccessCode": 200
    }
  },
  {
    "id": "5dca6cf302f380ec4350d522",
    "status": "active",
    "subject": {
      "entities": [
        {
          "idPattern": "delivery_robot_.*",
          "type": "delivery_robot"
        }
      ],
      "condition": {
        "attrs": [
          "mode"
        ]
      }
    },
    "notification": {
      "timesSent": 2,
      "lastNotification": "2019-11-12T08:28:19.00Z",
      "attrs": [
        "mode"
      ],
      "attrsFormat": "normalized",
      "http": {
        "url": "http://robot-controller:3000/api/v1/robots/notifications/"
      },
      "lastSuccess": "2019-11-12T08:28:19.00Z",
      "lastSuccessCode": 200
    }
  }
]
```